# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: ignored

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 66.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=acd34234780e83e44dd13b30e512c3d79d4811b4b470f0d9db8eeee84a7ec120
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

In [8]:
import pandas as pd
import numpy as np 
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [9]:
file = pd.read_csv('preprocessed_descriptions.csv')
word = set()
for i in range(len(file)):
  unique = []
  if type(file.preprocessed_descriptions[i]) == str:
    unique.append(word_tokenize(file.preprocessed_descriptions[i]))
  for j in range(len(unique)):
    word.update(unique[j])
word = list(word)
print(word[:5])

['albacore', 'scrolls', 'appetizercan', 'wichcraft', 'weave']


In [12]:
f2 = pd.read_csv('recipes_sample.csv')
recipes = f2.dropna(subset=['description'])

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [13]:
import random

In [14]:
for i in range(5):
  first = random.randint(0,len(word))
  second = random.randint(0,len(word))
  destination = nltk.edit_distance(word[first],word[second])
  print(f"Расстояние между парами слов {word[first]} и {word[second]} {destination} шагов")

Расстояние между парами слов bahamian и jaime 6 шагов
Расстояние между парами слов lpezalt и chances 7 шагов
Расстояние между парами слов triedjust и mdc 8 шагов
Расстояние между парами слов ottomans и impulsively 11 шагов
Расстояние между парами слов tried и 227 5 шагов


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [15]:
def Levenshtein(w,k,word):
  s = list()
  for i in word:
    s.append(tuple([nltk.edit_distance(w,i),i]))
  s.sort(key = lambda x: x[0])
  result = list()
  for i in s[:k]:
    result.append(i[1])
  return result
  

Levenshtein("sdfr", 4, word)

['seder', 'safer', 'dr', 'star']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
stemmer = nltk.SnowballStemmer('english')
lemmatiser = nltk.WordNetLemmatizer()

a,b = [],[]
for i in word:
  a.append(stemmer.stem(i))
  b.append(lemmatiser.lemmatize(i))
w1 = pd.DataFrame({'stemmed_word':a, 'normalized_word':b})
w1.index = word
w1


,stemmed_word,normalized_word
albacore,albacor,albacore
scrolls,scroll,scroll
appetizercan,appetizercan,appetizercan
wichcraft,wichcraft,wichcraft
weave,weav,weave
...,...,...
threeyearold,threeyearold,threeyearold
jackies,jacki,jackies
sued,su,sued
cookbook,cookbook,cookbook


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.corpus import stopwords
lst = []
for i in file['preprocessed_descriptions']:
    lst.extend(nltk.word_tokenize(str(i)))
print(len(lst))

stopw = stopwords.words("english")
lst2 = [x for x in lst if not x in stopw]
print(len(lst2))

from collections import Counter
print(f"Стоп-слова составили от общего количества слов {round((len(lst)-len(lst2))/len(lst), 5)}")
top1 = Counter(lst).most_common(10)
top2 = Counter(lst2).most_common(10)
print("Топ-10 самых часто употребляемых слов до удаления стоп-слов ", (top1))
print("Топ-10 самых часто употребляемых слов после удаления стоп-слов ", (top2))

1069885
581520
Стоп-слова составили от общего количества слов 0.45646
Топ-10 самых часто употребляемых слов до удаления стоп-слов  [('the', 40072), ('a', 34951), ('and', 30245), ('this', 26859), ('i', 24836), ('to', 23471), ('is', 20285), ('it', 19756), ('of', 18364), ('for', 15939)]
Топ-10 самых часто употребляемых слов после удаления стоп-слов  [('recipe', 14871), ('make', 6326), ('time', 5137), ('use', 4620), ('great', 4430), ('like', 4167), ('easy', 4152), ('one', 3872), ('made', 3810), ('good', 3791)]


In [ ]:
w_len = 0
w_st = 0
result = []
stop_words = stopwords.words('english')
for i in word:
  for j in nltk.word_tokenize(str(i)):
    w_len += 1
    if j not in stop_words:
      result.append(j)
    else:
      w_st += 1
w_st/w_len*100

0.0

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [22]:
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)

In [26]:
v = TfidfVectorizer()
file2 = pd.read_csv("recipes_sample.csv")
sample = file2.sample(5)
table = pd.DataFrame(v.fit_transform(sample.description).toarray(), columns=v.vocabulary_, index=sample.name)
table

,this,very,easy,and,delicious,skip,the,browning,use,dried,...,beautiful,refreshing,dessert,night,be,made,an,icecream,maker,don
name,,,,,,,,,,,,,,,,,,,,,
horseradish beef crock pot,0.000000,0.000000,0.00000,0.000000,0.475480,0.00000,0.00000,0.000000,0.000000,0.00000,...,0.190935,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000
healthy peanut butter honey cookies,0.000000,0.000000,0.13044,0.000000,0.174714,0.13044,0.13044,0.000000,0.000000,0.13044,...,0.105238,0.000000,0.000000,0.13044,0.13044,0.13044,0.087357,0.13044,0.315715,0.000000
orange sparklers,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.280229,0.00000,0.00000,0.00000,0.187673,0.00000,0.000000,0.280229
brandied cranberry sour cream cheesecake,0.212547,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,...,0.000000,0.212547,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000
the best vanilla lemon sorbet,0.000000,0.096216,0.00000,0.096216,0.128874,0.00000,0.00000,0.096216,0.096216,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.064437,0.00000,0.465758,0.000000


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [37]:
from scipy.spatial.distance import cosine

In [39]:
distanation = pd.DataFrame()
for i in table.iterrows():
    for j in table.iterrows():
        distanation.loc[i[0], j[0]] = cosine(i[1].values, j[1].values)
distanation

,brown sugar bbq chicken,moms tacos,orange vodka,garlic lime grilled chicken,apple dessert
brown sugar bbq chicken,0.000000,0.828669,0.927167,1.000000,0.884667
moms tacos,0.828669,0.000000,1.000000,1.000000,0.939597
orange vodka,0.927167,1.000000,0.000000,1.000000,0.899635
garlic lime grilled chicken,1.000000,1.000000,1.000000,0.000000,0.917308
apple dessert,0.884667,0.939597,0.899635,0.917308,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).